# YOLOv11-detection training
使用 Linux(WSL) 環境訓練，用以支援多 GPU 訓練 (nccl)

YOLO 訓練資訊: https://docs.ultralytics.com/tasks/segment/#train

In [1]:
import torch
from ultralytics import YOLO 


## 檢查運算單元

In [2]:
check_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用裝置: {check_device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")

使用裝置: cuda
GPU: NVIDIA TITAN V
GPU 記憶體: 12.0 GB


## 導入 YOLO 基礎模型
自動創建資料 `base_model` 並下載 YOLO 基礎模型

In [ ]:
# Load the YOLO base model
base_model = "yolo11l" # yolo11n.pt yolo11s.pt yolo11m.pt yolo11l.pt yolo11x.pt
model = YOLO(f"../base_model/detection/{base_model}.pt")

project_folder = "det_training_result"  
model_name = f"{base_model}-det_cow" 

# `Comet ml` 與 `ClearML` 擇一，不使用則 `pip uninstall clearml`
# Initialize Comet ML for tracking experiments
import comet_ml
# comet_ml.init() # set COMET_API_KEY
comet_ml.login(project_name=project_folder, 
               experiment_name=model_name,
               workspace="felimet", 
               api_key="fxwf3W0gSNOYWuOd67xpC1LSR")


COMET INFO: Valid Comet API Key saved in /home/isspmes/.comet.config (set COMET_CONFIG to change where it is saved).


### 自訂 Comet ML 日誌
[Comet ML 官方文件](https://www.comet.com/docs/v2/integrations/third-party-tools/yolov8/)

`Comet ml` 與 `ClearML` 擇一，不使用則 `pip uninstall clearml`

In [4]:
import os

# 控制 Comet ML 在實驗過程中記錄的圖像預測數量。預設情況下，Comet ML 會記錄來自驗證集的 100 個圖像預測。
os.environ["COMET_MAX_IMAGE_PREDICTIONS"] = "300"

# 指定記錄批次圖像預測的頻率。記錄 COMET_EVAL_BATCH_LOGGING_INTERVAL 環境變數控制此頻率。預設值為 1。
os.environ["COMET_EVAL_BATCH_LOGGING_INTERVAL"] = "1"

# 通過將 COMET_EVAL_LOG_CONFUSION_MATRIX 環境變數設置為 "false" 來停用混淆矩陣日誌。混淆矩陣只會在訓練完成後記錄一次。
# os.environ["COMET_EVAL_LOG_CONFUSION_MATRIX"] = "false"

# 離線記錄訓練結果
# os.environ['COMET_START_ONLINE'] = '0'  



## 訓練 YOLOv11-det
訓練完成的模型與結果儲存於 `training_result_yolo11l_det` 資料夾中

In [ ]:
# import torch
# import gc

# # 清空CUDA快取
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.synchronize()

# # 強制垃圾回收
# gc.collect()

# import os
# os._exit(00)

In [ ]:
# 訓練
# 詳細參數見 https://docs.ultralytics.com/zh/tasks/detect/
# https://docs.ultralytics.com/zh/usage/cfg/#train-settings

results = model.train(
    data="../dataset_det/yolo_data_integrated/dataset.yaml",
    epochs=130,
    imgsz=640,
    device=[0, 1] if check_device == "cuda" else "cpu", 
    batch=32 if check_device == "cuda" else 16, 
    workers=8 if check_device == "cuda" else 4, 
    patience=20, 
    cache="disk" if check_device == "cuda" else False,
    amp=True if check_device == "cuda" else False,
    verbose=True,
    plots=True,
    save_period=30,
    save=True,  
    save_frames=True,
    save_json=True,
    pretrained=True,
    project=project_folder,
    name=model_name,
    exist_ok=True, # 允許覆蓋現有項目
    optimizer="auto",  
    mosaic=0.9,    
    cutmix=0.5,
    mixup=0.4,
    copy_paste=0.2,
    lr0=0.008, 
    lrf=0.01,  
    weight_decay=8e-4, # L2 正則化
    translate=0.15,
    scale=0.4,    
)

print("訓練完成！")
print(f"訓練結果儲存於: {project_folder}/{model_name}")

New https://pypi.org/project/ultralytics/8.3.155 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.154 🚀 Python-3.12.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN V, 12288MiB)
                                                        CUDA:1 (NVIDIA TITAN V, 12287MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.5, data=./dataset_det/yolo_data_integrated/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=130, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.4, mode=train, model=b

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Experiment is live on comet.com https://www.comet.com/felimet/training-result-yolo11l-det/bdc45452981c4a66bb83b4245e2d9aa0



TensorBoard: Start with 'tensorboard --logdir training_result_yolo11l-det/yolo11l-det_cow', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1
Transferred 1009/1015 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 141.4±22.3 MB/s, size: 61.4 KB)


train: Scanning /home/isspmes/YOLOHub/dataset_det/yolo_data_integrated/labels/train.cache... 661 images, 0 backgrounds, 0 corrupt: 100%|██████████| 661/661 [00:00<?, ?it/s]
train: Caching images (0.6GB Disk): 100%|██████████| 661/661 [00:00<00:00, 12275.99it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 84.9±40.4 MB/s, size: 62.5 KB)


...

                   all        189        356      0.997      0.992      0.995      0.977
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 95, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

115 epochs completed in 0.450 hours.
Optimizer stripped from training_result_yolo11l-det/yolo11l-det_cow/weights/last.pt, 51.2MB
Optimizer stripped from training_result_yolo11l-det/yolo11l-det_cow/weights/best.pt, 51.2MB

Validating training_result_yolo11l-det/yolo11l-det_cow/weights/best.pt...
Ultralytics 8.3.154 🚀 Python-3.12.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN V, 12288MiB)
                                                        CUDA:1 (NVIDIA TITAN V, 12287MiB)
YOLO11l summary (fused): 190 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.21it/s]


                   all        189        356      0.994      0.992      0.995      0.977
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 2.7ms postprocess per image
Saving training_result_yolo11l-det/yolo11l-det_cow/predictions.json...
Results saved to training_result_yolo11l-det/yolo11l-det_cow


COMET ERROR: We failed to read file 'training_result_yolo11l-det/yolo11l-det_cow/labels_correlogram.jpg' for uploading.
Please double-check the file path and permissions
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : yolo11l-det_cow
COMET INFO:     url                   : https://www.comet.com/felimet/training-result-yolo11l-det/bdc45452981c4a66bb83b4245e2d9aa0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [231]               : (0.0002636923076923077, 0.0019390769230769231)
COMET INFO:     lr/pg1 [231]               : (0.0002636923076923077, 0.0019390769230769231)
COMET INFO:     lr/pg2 [231]               : (0.0002636923076923077, 0.0019390769230769231)
COMET INFO:     metrics

訓練完成！
訓練結果儲存於: training_result_yolo11l-det/yolo11l-det_cow
